In [4]:
import datetime
import MetaTrader5 as mt5
import numpy as np
import pandas as pd
from datetime import datetime as dt
import numba as nb
from collections import defaultdict 
from tqdm.notebook import tqdm

In [5]:
if not mt5.initialize():
    print(mt5.last_error())
    mt5.shutdown()
else:
    authorized=mt5.login(18676, password="aTOirs42", server="GoMarkets-Demo")
    print(mt5.last_error())

(1, 'Success')


In [17]:
def getData(numBefore, timePeriod, signal, timeDict):
    rates = mt5.copy_rates_from_pos(signal, timeDict[timePeriod], 1, numBefore)       
    ratesDf = pd.DataFrame(rates)
    return ratesDf

def getStoch(data, n):
    close = data.close
    low = data.low
    high = data.high
    
    STOK = ((close - low.rolling(n).min()) / (high.rolling(n).max() - low.rolling(n).min())) * 100
    STOD = STOK.rolling(3).mean()
    
    return STOK, STOD

def getAllInfo(rates, n):
    stok, stod = getStoch(rates, n)
    
    outDf = rates.copy()
    outDf['stok'] = stok
    outDf['stod'] = stod
    
    outDf.drop(columns=['tick_volume', 'spread', 'real_volume'], inplace=True)
    finalOut = outDf[outDf['stod']>0]
    
    return finalOut

## Try to simply get the value of the scholastic for a signal

In [31]:
## Get the values and compare with real data to validate
timeDict = {'15': mt5.TIMEFRAME_M15, '30': mt5.TIMEFRAME_M30, '60': mt5.TIMEFRAME_H1}
timePeriod = '15'

# # Get the stochastic df
rates = getData(100, timePeriod, 'AUDUSD', timeDict) 
filteredDf = getAllInfo(rates, 14)
print(filteredDf)

          time     open     high      low    close       stok       stod
15  1604999700  0.72848  0.72927  0.72848  0.72919  95.428571  83.781221
16  1605000600  0.72920  0.72929  0.72797  0.72860  61.016949  75.701401
17  1605001500  0.72859  0.72881  0.72755  0.72756   2.259887  52.901803
18  1605002400  0.72754  0.72876  0.72749  0.72834  47.222222  36.833019
19  1605003300  0.72835  0.72892  0.72804  0.72856  59.444444  36.308851
..         ...      ...      ...      ...      ...        ...        ...
95  1605071700  0.73103  0.73180  0.73092  0.73174  98.584906  86.903215
96  1605072600  0.73176  0.73176  0.73092  0.73125  87.028302  91.767976
97  1605073500  0.73122  0.73176  0.73095  0.73130  88.207547  91.273585
98  1605074400  0.73129  0.73142  0.73065  0.73078  75.943396  83.726415
99  1605075300  0.73074  0.73086  0.73061  0.73065  72.877358  79.009434

[85 rows x 7 columns]


In [36]:
#Thresholds for schocastic
threshUp = 80
threshDown = 20

# Get the signals required
# signals = mt5.symbols_get(group="*GBP*, *JPY*, *EUR*, *AUD*, *GBP*, *CHF*, *NZD*, *CAD*")
signals = mt5.symbols_get(group='*GBP*')
sigNames = [sig.name for sig in signals]
print(sigNames)

['GBPUSD', 'EURGBP', 'GBPCHF', 'GBPJPY', 'GBPAUD', 'GBPCAD', 'GBPNZD']


In [27]:
# sigNames = sigNames[0:3]
# sigNames

['EURUSD', 'GBPUSD', 'USDCHF']

In [39]:
# Set up the variables required
barsAfter = [3, 5, 7]
resultDict = {}
pips = 0.0005

In [40]:
for sig in sigNames:
    print(sig)
    
    for key, val in tqdm(timeDict.items()):
        print(key)

        resultDict[key] = defaultdict(list)

        numYears = 0.1
        numBefore = int(60/int(key) * 24 * 365 * numYears) # Data for one year 
        print(numBefore)
        
        rates = getData(numBefore, key, sig, timeDict) 
        filteredDf = getAllInfo(rates, 14)
        print(mt5.last_error())       
        
        ## CHECK IF THERE IS A WIN AFTER BREACHING STOCHASTIC:
        for index, row in tqdm(filteredDf.iterrows()):
            if index > len(filteredDf)-max(barsAfter)-1:
                 continue

            # Current bar has closed outside upper threshold
            if row.stok > threshUp or row.stod > threshUp:
#                 print(pd.to_datetime(row.time, unit='s'))
                for n in barsAfter:
                    nextBars = filteredDf.iloc[index+1:index+n+1]

                    if min(nextBars.low) < (row.close - pips):
                        resultDict[key][n].append(True)
                    else:
                        resultDict[key][n].append(False)

            elif row.stok < threshDown or row.stod < threshDown:

                for n in barsAfter:
                    nextBars = filteredDf.iloc[index+1:index+n+1]

                    if max(nextBars.high) > (row.close + pips):
                        resultDict[key][n].append(True)
                    else:
                        resultDict[key][n].append(False)

#     print('timeframe', 'bars_after', 'truth_values', 'percentage_true')
    for key, val in resultDict.items():
        for i, j in val.items():
            n, c = np.unique(j, return_counts=True)
            print(key, i, n, c, c[1]/sum(c)*100)
        print('\n')

GBPUSD


15
3504
(1, 'Success')



30
1752
(1, 'Success')



60
876
(1, 'Success')




15 3 [False  True] [ 669 1040] 60.85430076067876
15 5 [False  True] [ 599 1110] 64.95026331187829
15 7 [False  True] [ 547 1162] 67.99297834991222


30 3 [False  True] [306 538] 63.74407582938388
30 5 [False  True] [288 556] 65.87677725118483
30 7 [False  True] [271 573] 67.89099526066352


60 3 [False  True] [137 281] 67.22488038277513
60 5 [False  True] [116 302] 72.24880382775119
60 7 [False  True] [ 98 320] 76.55502392344498


EURGBP


...



15 3 [False  True] [606 997] 62.19588271990018
15 5 [False  True] [ 554 1049] 65.4398003742982
15 7 [False  True] [ 519 1084] 67.6232064878353


30 3 [False  True] [318 450] 58.59375
30 5 [False  True] [297 471] 61.328125
30 7 [False  True] [283 485] 63.151041666666664


60 3 [False  True] [105 269] 71.92513368983957
60 5 [False  True] [ 92 282] 75.40106951871658
60 7 [False  True] [ 88 286] 76.47058823529412


GBPNZD


15
3504
(1, 'Success')



30
1752
(1, 'Success')



60
876
(1, 'Success')




15 3 [False  True] [599 976] 61.968253968253975
15 5 [False  True] [ 529 1046] 66.41269841269842
15 7 [False  True] [ 472 1103] 70.03174603174604


30 3 [False  True] [332 438] 56.88311688311688
30 5 [False  True] [304 466] 60.51948051948052
30 7 [False  True] [284 486] 63.116883116883116


60 3 [False  True] [139 252] 64.45012787723785
60 5 [False  True] [128 263] 67.26342710997443
60 7 [False  True] [119 272] 69.56521739130434


